# Entraînement baseline du modèle CNN
Ce notebook présente :
- Chargement des données prétraitées
- Définition d’un modèle CNN simple
- Entraînement et évaluation de la baseline
- Sauvegarde du meilleur modèle

## Imports & Paramètres

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Répertoires
DATA_DIR = "../data/processed"
CHECKPOINT_DIR = "../outputs/checkpoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# Hyperparamètres
IMG_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 1e-3

## Chargement des données

In [ ]:
datagen = ImageDataGenerator(validation_split=0.2)

train_gen = datagen.flow_from_directory(
    DATA_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode="categorical", subset="training", shuffle=True
)
val_gen = datagen.flow_from_directory(
    DATA_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode="categorical", subset="validation", shuffle=False
)

## Définition du modèle

In [ ]:
def build_baseline_cnn(input_shape, num_classes):
    model = models.Sequential([
        tf.keras.Input(shape=input_shape),
        layers.Conv2D(32, 3, activation="relu", padding="same"),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation="relu", padding="same"),
        layers.MaxPooling2D(),
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax")
    ])
    return model

model = build_baseline_cnn(IMG_SIZE + (3,), len(train_gen.class_indices))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

## Callbacks et entraînement

In [ ]:
checkpoint_cb = callbacks.ModelCheckpoint(
    os.path.join(CHECKPOINT_DIR, "baseline_model.keras"),
    save_best_only=True, monitor="val_accuracy", mode="max"
)
earlystop_cb = callbacks.EarlyStopping(
    monitor="val_loss", patience=5, restore_best_weights=True
)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=[checkpoint_cb, earlystop_cb]
)

## Visualisation des courbes

In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history["accuracy"], label="Train Acc")
plt.plot(history.history["val_accuracy"], label="Val Acc")
plt.legend(); plt.title("Accuracy")

plt.subplot(1,2,2)
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Val Loss")
plt.legend(); plt.title("Loss")
plt.show()

## Sauvegarde finale et exemple de prédiction

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
from PIL import Image

In [ ]:
best_model = load_model(os.path.join(CHECKPOINT_DIR, "baseline_model.keras"))

# Exemple prédiction
img = Image.open("../data/processed/<classe>/<exemple>.jpg").resize(IMG_SIZE)
x = np.array(img) / 255.0
pred = best_model.predict(x[np.newaxis])
print("Classe prédite :", train_gen.class_indices, "->", np.argmax(pred))
